In [1]:
from datasets import load_dataset
import pandas as pd
mbpp = load_dataset("mbpp")

No config specified, defaulting to: mbpp/full
Found cached dataset mbpp (/home/chloe/.cache/huggingface/datasets/mbpp/full/1.0.2/4458a31cd4305553c8e88e3f0bfb94fc74fe1a9faeeb8c32ed166939735eaeff)


  0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
mbpp_train_df = pd.DataFrame(mbpp["train"])
mbpp_train_df.describe()

,task_id
count,374.000000
mean,787.500000
std,108.108742
min,601.000000
25%,694.250000
50%,787.500000
75%,880.750000
max,974.000000


In [4]:
mbpp_prompt_df = pd.DataFrame(mbpp["prompt"])
mbpp_test_df = pd.DataFrame(mbpp["test"])
print(mbpp_prompt_df.iloc[3]['text'])
print(mbpp_test_df.iloc[3]['test_list'])

Write a function to find the largest integers from a given list of numbers using heap queue algorithm.
['assert find_Volume(10,8,6) == 240', 'assert find_Volume(3,2,2) == 6', 'assert find_Volume(1,2,1) == 1']


In [5]:
mbpp_test_df.describe()

,task_id
count,500.000000
mean,260.500000
std,144.481833
min,11.000000
25%,135.750000
50%,260.500000
75%,385.250000
max,510.000000


In [19]:
max_list_length = mbpp_test_df['test_list'].apply(len).max()
max_list_length

3

In [24]:
# format the test df such that it uses the same format as the coqllm dataset
mbpp_test_df[['test_case_1', 'test_case_2', 'test_case_3']] = mbpp_test_df['test_list'].apply(
    lambda x: pd.Series(x[:3] + [None] * max(0, 3 - len(x)))
)
mbpp_test_coq = mbpp_test_df[['text', 'test_case_1', 'test_case_2', 'test_case_3']]
mbpp_test_coq = mbpp_test_coq.rename(columns={'text': 'specification'})
# remove "assert" statements from the test cases
test_case_columns = ['test_case_1', 'test_case_2', 'test_case_3']
def remove_asserts(text):
    if text is None:
        return None
    return text.replace('assert ', '')
def fix_equals(text):
    if text is None:
        return None
    return text.replace('==', '=')
def change_language(text):
    if text is None:
        return None
    return text.replace('Write a python function', 'Write a Coq function')

mbpp_test_coq[test_case_columns] = mbpp_test_coq[test_case_columns].applymap(remove_asserts)
mbpp_test_coq[test_case_columns] = mbpp_test_coq[test_case_columns].applymap(fix_equals)
mbpp_test_coq[['specification']] = mbpp_test_coq[['specification']].applymap(change_language)

mbpp_test_coq.head()


,specification,test_case_1,test_case_2,test_case_3
0,Write a Coq function to remove first and last ...,"remove_Occ(""hello"",""l"") = ""heo""","remove_Occ(""abcda"",""a"") = ""bcd""","remove_Occ(""PHP"",""P"") = ""H"""
1,Write a function to sort a given matrix in asc...,"sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])...","sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1...","sort_matrix([[5,8,9],[6,4,3],[2,1,4]])=[[2, 1,..."
2,Write a function to count the most common word...,"count_common(['red','green','black','pink','bl...","count_common(['one', 'two', 'three', 'four', '...","count_common(['Facebook', 'Apple', 'Amazon', '..."
3,Write a Coq function to find the volume of a t...,"find_Volume(10,8,6) = 240","find_Volume(3,2,2) = 6","find_Volume(1,2,1) = 1"
4,Write a function to split a string at lowercas...,"split_lowerstring(""AbCd"")=['bC','d']","split_lowerstring(""Python"")=['y', 't', 'h', 'o...","split_lowerstring(""Programming"")=['r', 'o', 'g..."


In [25]:
mbpp_test_coq.to_csv('MBPP_Coq_Test.csv', index=False)